In [6]:
# 모듈 import
import pandas as pd
import numpy as np

from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [3]:
# path 지정
base_path = "/content/drive/MyDrive/Colab Notebooks/data/235687_시스템 품질 변화로 인한 사용자 불편 예지 AI 경진대회_data/convert/"
save_path = "/content/drive/MyDrive/Colab Notebooks/data/235687_시스템 품질 변화로 인한 사용자 불편 예지 AI 경진대회_data/result/"

In [5]:
# train data 불러오기
train_quality = pd.read_csv(base_path+'train_quality_data_.csv')
train_problem = pd.read_csv(base_path+'train_problem_data_.csv')
train_error = pd.read_csv(base_path+'train_err_data_.csv')
test_quality = pd.read_csv(base_path+'test_quality_data_.csv')
test_error = pd.read_csv(base_path+'test_err_data_.csv')

In [7]:
# train_error
id_error = train_error[['user_id', 'errtype']].values
error = np.zeros((15000,42))

for person_idx, err in tqdm(id_error):
    # person_idx -10000 위치에 person_idx, errtype에 해당하는 error값을 +1
    error[person_idx - 10000, err - 1] += 1

100%|██████████| 16554663/16554663 [00:47<00:00, 351706.37it/s]


In [8]:
train_df = pd.DataFrame(error)

In [9]:
# train_problem
problem = np.zeros(15000)
problem[train_problem.user_id.unique()-10000] = 1

In [10]:
target = pd.DataFrame(problem)

In [13]:
lr_clf = LogisticRegression()
knn_clf = KNeighborsClassifier(n_neighbors=8)

vo_clf = VotingClassifier(estimators=[('LR', lr_clf), ('KNN', knn_clf)], voting='soft')
# train validation 분리
X_train, X_val, y_train, y_val = train_test_split(train_df,target,test_size=0.2, random_state=42)

vo_clf.fit(X_train, y_train)
pred = vo_clf.predict(X_val)
print(f'보팅 분류기 정확도: {np.round(accuracy_score(y_val, pred), 4)}')

classifiers = [lr_clf, knn_clf]
for classifier in classifiers:
    classifier.fit(X_train, y_train)
    pred = classifier.predict(X_val)
    class_name = classifier.__class__.__name__
    print('{0} 정확도: {1:.4f}'.format(class_name, accuracy_score(y_val, pred)))

보팅 분류기 정확도: 0.585
LogisticRegression 정확도: 0.5690
KNeighborsClassifier 정확도: 0.5840
